In [1]:
import openpyxl
import pandas as pd
import os
import datetime
#from IPython.display import display
import copy

In [2]:
def print_full(x):
    df = pd.DataFrame(x)
    pd.set_option('display.max_rows', 2000)
    pd.set_option('display.max_columns', 20)
    display(df)

### import and export excel 

In [3]:
excelname= "ma"
sheetname = "10ma"

wb = openpyxl.load_workbook(str(excelname)+".xlsx")
ws = wb[str(sheetname)]

dataset = []

i = 1
while(ws.cell(row=i, column=1).value!=None):
    row = []
    row.append(ws.cell(row=i, column=1).value)
    row.append(ws.cell(row=i, column=2).value)
    dataset.append(row)
    i = i + 1
pd.DataFrame(dataset).head()

,0,1
0,date,closeprice
1,2003-07-01,38.05
2,2003-07-02,38.69
3,2003-07-03,39
4,2003-07-04,39.26


In [27]:
pd.DataFrame(dataset).head()

,0,1
0,date,closeprice
1,2003-07-01,38.05
2,2003-07-02,38.69
3,2003-07-03,39
4,2003-07-04,39.26


In [28]:
ma_index

10

In [45]:
def hyper(dataset, ma_index):
    trade = copy.deepcopy(dataset)
    for i in range(0, len(trade)):
        trade[i].append(0)
        trade[i].append(0)
        trade[i].append(0)
        trade[i].append(0)
        trade[i].append(0)
        trade[i].append(0)
        trade[i].append(0)
        trade[i].append(0)
        trade[i].append(0)    
    trade[0][2] = "10ma"
    trade[0][3] = "buy"
    trade[0][4] = "hold"
    trade[0][5] = "sell"
    trade[0][6] = "number"
    trade[0][7] = "value"
    trade[0][8] = "period_return"
    trade[0][9] = "holding days"
    trade[0][10] = "annual return"

    for i in range(ma_index, len(trade)):
        temp = 0
        for j in range(0, ma_index):
            temp = temp + trade[i-j][1]
        trade[i][2] = temp / ma_index
    pd.DataFrame(trade).head(20)    

    for i in range(ma_index, len(trade)):
        if(trade[i][1] > trade[i][2])and(trade[i-1][4]==0):
            trade[i][3] = 1
            trade[i][4] = 1
            trade[i][5] = 0
        elif(trade[i][1] < trade[i][2])and(trade[i-1][4]==1):
            trade[i][3] = 0
            trade[i][4] = 0
            trade[i][5] = 1
        elif(trade[i-1][4]>0):
            trade[i][3] = 0
            trade[i][4] = 1
            trade[i][5] = 0    

    initial = 10000
    for i in range(ma_index, len(trade)):
        if(trade[i][3] == 1)and(trade[i][4]==1)and(trade[i][5] == 0):
            trade[i][6] = initial/trade[i][1]
            trade[i][7] = trade[i][6] * trade[i][1]
            trade[i][8] = round((trade[i][6] * trade[i][1] - initial)/initial, 4)
        elif(trade[i][3] == 0)and(trade[i][4]==0)and(trade[i][5] == 1):
            trade[i][6] = 0
            trade[i][7] = trade[i-1][6] * trade[i][1]
            trade[i][8] = round((trade[i-1][6] * trade[i][1] - initial)/initial, 4)
        elif(trade[i][3] == 0)and(trade[i][4]==1)and(trade[i][5] == 0):
            trade[i][6] = trade[i-1][6]
            trade[i][7] = trade[i][6] * trade[i][1]
            trade[i][8] = round((trade[i][6] * trade[i][1] - initial)/initial, 4)            
    start = 0
    end = 0
    for i in range(1, len(trade)):
        if(trade[i][3]==1):
            trade[i][9] = 1
            start = datetime.datetime.strptime(trade[i][0], "%Y-%m-%d").date()
        elif((trade[i][4] > 0) or (trade[i][5] == 1))and(trade[i][3] == 0):
            end = datetime.datetime.strptime(trade[i][0], "%Y-%m-%d").date()
            delta = end - start
            trade[i][9] = delta.days+1
            
    for i in range(1, len(trade)):
        if(trade[i][3]==1)or(trade[i][4]==1)or(trade[i][5]==1):
            trade[i][10] = (1+trade[i][8])**(365/trade[i][9])-1
            
    # annual return computation
    # https://www.investopedia.com/terms/a/annualized-total-return.asp
    annual_return = 1
    trading_counts = 0
    for i in range(1, len(trade)):
        if(trade[i][5]==1):
            annual_return = annual_return * (1+trade[i][8])
            trading_counts = trading_counts + 1
#     print(annual_return)
#     print(trading_counts)
    final_return = annual_return**(1/trading_counts)-1
#     print(final_return)
    return final_return

In [47]:
hyper(dataset, 10)

-8.316635349714385e-05

In [52]:
for i in range(2, 10):
    print("Thin final annual return of ma" + str(i)+" is:" + str(hyper(dataset, i)))

Thin final annual return of ma2 is:0.0006106951900699542
Thin final annual return of ma3 is:-0.00016588926514460312
Thin final annual return of ma4 is:-0.00016331547396630786
Thin final annual return of ma5 is:-0.0005151006523820278
Thin final annual return of ma6 is:-0.000818800676472331
Thin final annual return of ma7 is:-0.0003124742224202004
Thin final annual return of ma8 is:-0.0008119553272980529
Thin final annual return of ma9 is:-1.4384400451761259e-05


In [46]:
hyper(dataset, 10)

-8.316635349714385e-05

In [42]:
# pd.DataFrame(hyper(dataset, 5)).head(40)

,0,1,2,3,4,5,6,7,8,9,10
0,date,closeprice,10ma,buy,hold,sell,number,value,period_return,holding days,annual return
1,2003-07-01,38.05,0,0,0,0,0,0,0,0,0
2,2003-07-02,38.69,0,0,0,0,0,0,0,0,0
3,2003-07-03,39,0,0,0,0,0,0,0,0,0
4,2003-07-04,39.26,0,0,0,0,0,0,0,0,0
5,2003-07-07,41,39.2,1,1,0,243.902,10000,0,1,0
6,2003-07-08,41.19,39.828,0,1,0,243.902,10046.3,0.0046,2,1.31076
7,2003-07-09,41.22,40.334,0,1,0,243.902,10053.7,0.0054,3,0.92559
8,2003-07-10,40.05,40.544,0,0,1,0,9768.29,-0.0232,4,-0.882574
9,2003-07-11,39.91,40.674,0,0,0,0,0,0,0,0


In [4]:
trade = copy.deepcopy(dataset)
pd.DataFrame(trade).head()

,0,1
0,date,closeprice
1,2003-07-01,38.05
2,2003-07-02,38.69
3,2003-07-03,39
4,2003-07-04,39.26


In [5]:
for i in range(0, len(trade)):
    trade[i].append(0)
    trade[i].append(0)
    trade[i].append(0)
    trade[i].append(0)
    trade[i].append(0)
    trade[i].append(0)
    trade[i].append(0)
    trade[i].append(0)
    trade[i].append(0)    
pd.DataFrame(trade).head()

,0,1,2,3,4,5,6,7,8,9,10
0,date,closeprice,0,0,0,0,0,0,0,0,0
1,2003-07-01,38.05,0,0,0,0,0,0,0,0,0
2,2003-07-02,38.69,0,0,0,0,0,0,0,0,0
3,2003-07-03,39,0,0,0,0,0,0,0,0,0
4,2003-07-04,39.26,0,0,0,0,0,0,0,0,0


In [6]:
trade[0][2] = "10ma"
trade[0][3] = "buy"
trade[0][4] = "hold"
trade[0][5] = "sell"
trade[0][6] = "number"
trade[0][7] = "value"
trade[0][8] = "period_return"
trade[0][9] = "holding days"
trade[0][10] = "annual return"
pd.DataFrame(trade).head()

,0,1,2,3,4,5,6,7,8,9,10
0,date,closeprice,10ma,buy,hold,sell,number,value,period_return,holding days,annual return
1,2003-07-01,38.05,0,0,0,0,0,0,0,0,0
2,2003-07-02,38.69,0,0,0,0,0,0,0,0,0
3,2003-07-03,39,0,0,0,0,0,0,0,0,0
4,2003-07-04,39.26,0,0,0,0,0,0,0,0,0


In [7]:
ma_index = 10
for i in range(ma_index, len(trade)):
    temp = 0
    for j in range(0, ma_index):
        temp = temp + trade[i-j][1]
    trade[i][2] = temp / ma_index
pd.DataFrame(trade).head(20)

,0,1,2,3,4,5,6,7,8,9,10
0,date,closeprice,10ma,buy,hold,sell,number,value,period_return,holding days,annual return
1,2003-07-01,38.05,0,0,0,0,0,0,0,0,0
2,2003-07-02,38.69,0,0,0,0,0,0,0,0,0
3,2003-07-03,39,0,0,0,0,0,0,0,0,0
4,2003-07-04,39.26,0,0,0,0,0,0,0,0,0
5,2003-07-07,41,0,0,0,0,0,0,0,0,0
6,2003-07-08,41.19,0,0,0,0,0,0,0,0,0
7,2003-07-09,41.22,0,0,0,0,0,0,0,0,0
8,2003-07-10,40.05,0,0,0,0,0,0,0,0,0
9,2003-07-11,39.91,0,0,0,0,0,0,0,0,0


In [8]:
for i in range(ma_index, len(trade)):
    if(trade[i][1] > trade[i][2])and(trade[i-1][4]==0):
        trade[i][3] = 1
        trade[i][4] = 1
        trade[i][5] = 0
    elif(trade[i][1] < trade[i][2])and(trade[i-1][4]==1):
        trade[i][3] = 0
        trade[i][4] = 0
        trade[i][5] = 1
    elif(trade[i-1][4]>0):
        trade[i][3] = 0
        trade[i][4] = 1
        trade[i][5] = 0

In [11]:
pd.DataFrame(trade).head(3)

,0,1,2,3,4,5,6,7,8,9,10
0,date,closeprice,10ma,buy,hold,sell,number,value,period_return,holding days,annual return
1,2003-07-01,38.05,0,0,0,0,0,0,0,0,0
2,2003-07-02,38.69,0,0,0,0,0,0,0,0,0


In [12]:
initial = 10000
for i in range(ma_index, len(trade)):
    if(trade[i][3] == 1)and(trade[i][4]==1)and(trade[i][5] == 0):
        trade[i][6] = initial/trade[i][1]
        trade[i][7] = trade[i][6] * trade[i][1]
        trade[i][8] = round((trade[i][6] * trade[i][1] - initial)/initial, 4)
    elif(trade[i][3] == 0)and(trade[i][4]==0)and(trade[i][5] == 1):
        trade[i][6] = 0
        trade[i][7] = trade[i-1][6] * trade[i][1]
        trade[i][8] = round((trade[i-1][6] * trade[i][1] - initial)/initial, 4)
    elif(trade[i][3] == 0)and(trade[i][4]==1)and(trade[i][5] == 0):
        trade[i][6] = trade[i-1][6]
        trade[i][7] = trade[i][6] * trade[i][1]
        trade[i][8] = round((trade[i][6] * trade[i][1] - initial)/initial, 4)

In [16]:
pd.DataFrame(trade).head(5)

,0,1,2,3,4,5,6,7,8,9,10
0,date,closeprice,10ma,buy,hold,sell,number,value,period_return,holding days,annual return
1,2003-07-01,38.05,0,0,0,0,0,0,0,0,0
2,2003-07-02,38.69,0,0,0,0,0,0,0,0,0
3,2003-07-03,39,0,0,0,0,0,0,0,0,0
4,2003-07-04,39.26,0,0,0,0,0,0,0,0,0


In [18]:
start = 0
end = 0
for i in range(1, len(trade)):
    if(trade[i][3]==1):
        trade[i][9] = 1
        start = datetime.datetime.strptime(trade[i][0], "%Y-%m-%d").date()
    elif((trade[i][4] > 0) or (trade[i][5] == 1))and(trade[i][3] == 0):
        end = datetime.datetime.strptime(trade[i][0], "%Y-%m-%d").date()
        delta = end - start
        trade[i][9] = delta.days+1
# pd.DataFrame(trade)

In [20]:
for i in range(1, len(trade)):
    if(trade[i][3]==1)or(trade[i][4]==1)or(trade[i][5]==1):
        trade[i][10] = (1+trade[i][8])**(365/trade[i][9])-1
# pd.DataFrame(trade)

In [23]:
# annual return computation
# https://www.investopedia.com/terms/a/annualized-total-return.asp
annual_return = 1
trading_counts = 0
for i in range(1, len(trade)):
    if(trade[i][5]==1):
        annual_return = annual_return * (1+trade[i][8])
        trading_counts = trading_counts + 1
print(annual_return)
print(trading_counts)
final_return = annual_return**(1/trading_counts)-1
print(final_return)

0.9765753225396776
285
-8.316635349714385e-05
